In [1]:
import os
ROOT = "/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/DLCVNLP/Live Lectures Material DLCVNLP April 17th batch/ALL NOTES/Day40 Sep 4th (PYTORCH 2nd class)"

os.chdir(ROOT)
os.getcwd()

'/content/drive/MyDrive/001_SUNNY_BHAVEEN_CHANDRA/001_My_Classes/DLCVNLP/Live Lectures Material DLCVNLP April 17th batch/ALL NOTES/Day40 Sep 4th (PYTORCH 2nd class)'

In [2]:
ZIPFILE = "./hymenoptera_data.zip"

In [3]:
from zipfile import ZipFile

In [4]:
# with ZipFile(ZIPFILE) as f:
#   f.extractall()

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [6]:
data_dir = "./hymenoptera_data"

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# transforms

data_transform = {
    "train": transforms.Compose([
                                 transforms.RandomSizedCrop(224),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor()
    ]),
    "val": transforms.Compose([
                                 transforms.Resize(256),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()
    ]) 
}

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:917: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [9]:
# image_dataset = {
#     "train": datasets.ImageFolder(os.path.join(data_dir, "train")),
#     "val": datasets.ImageFolder(os.path.join(data_dir, "val"))
# }

image_dataset = {
    "train": datasets.ImageFolder(os.path.join(data_dir, "train"),transform=data_transform['train']), # mod
    "val": datasets.ImageFolder(os.path.join(data_dir, "val"),transform=data_transform['val']) # mod
}

In [10]:
dataLoader = {
    "train": torch.utils.data.DataLoader(image_dataset["train"], batch_size = 4, shuffle=True, num_workers=4),
    "val": torch.utils.data.DataLoader(image_dataset["val"], batch_size = 4, shuffle=True, num_workers=4)
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
data_sizes = {
    "train": len(image_dataset["train"]),
    "val": len(image_dataset["val"])
}
data_sizes

{'train': 244, 'val': 153}

In [12]:
class_names = image_dataset["train"].classes
class_names

['ants', 'bees']

In [13]:
model_conv = torchvision.models.resnet18(pretrained=True)

print(model_conv)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
# freezing the conv layers
for param in model_conv.parameters():
  param.required_grad = False

In [15]:
num_ftrs = model_conv.fc.in_features

num_ftrs

512

In [16]:
model_conv.fc

Linear(in_features=512, out_features=1000, bias=True)

In [17]:
no_of_classes = len(class_names)

model_conv.fc = nn.Linear(num_ftrs, no_of_classes) # incomming, outgoing edges 


In [18]:
print(model_conv)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
model_conv = model_conv.to(device)

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.fc.parameters(), lr=0.001)

In [21]:
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [24]:
def train_loop(model, loss_fn, optimizer, scheduler, num_epochs=20):

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
    print(f"Epoch {epoch}/{num_epochs - 1}")
    print("=="*10)

    for phase in ["train", "val"]:
      training = phase == "train"
      if training:
        model.train() # sets the training mode
      else:
        model.eval() # sets the evaluation mode

      running_loss = 0.0
      running_corrects = 0

      for inputs, labels in dataLoader[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(training):
          outputs = model(inputs)

          _, preds = torch.max(outputs, 1)
          loss = loss_fn(outputs, preds)

          if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

      if training:
        scheduler.step()

      epoch_loss = running_loss / data_sizes[phase]
      epoch_acc = running_corrects.double() / data_sizes[phase] # mod

      print(f"{phase}, Loss: {epoch_loss}, accuracy: {epoch_acc}")
      if not training and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
  
  model.load_state_dict(best_model_wts)
  return model


        




In [25]:
model = train_loop(model_conv, loss_fn, optimizer, exp_lr_scheduler, num_epochs=20)

Epoch 0/19


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


train, Loss: 7.532173424165269e-05, accuracy: 0.49590163934426235
val, Loss: 0.0001265768638654217, accuracy: 0.542483660130719
Epoch 1/19
train, Loss: 7.000964692675462e-05, accuracy: 0.49590163934426235
val, Loss: 0.00010131899847795049, accuracy: 0.542483660130719
Epoch 2/19
train, Loss: 6.210234944580302e-05, accuracy: 0.49590163934426235
val, Loss: 8.925120924342004e-05, accuracy: 0.542483660130719
Epoch 3/19
train, Loss: 9.394090664530073e-05, accuracy: 0.49590163934426235
val, Loss: 0.00010269415531118378, accuracy: 0.542483660130719
Epoch 4/19
train, Loss: 0.00014609664509914526, accuracy: 0.49590163934426235
val, Loss: 6.327511158922468e-05, accuracy: 0.542483660130719
Epoch 5/19
train, Loss: 6.047579960873232e-05, accuracy: 0.49590163934426235
val, Loss: 6.512218553824841e-05, accuracy: 0.542483660130719
Epoch 6/19
train, Loss: 7.298129402756114e-05, accuracy: 0.49590163934426235
val, Loss: 7.610039690417562e-05, accuracy: 0.542483660130719
Epoch 7/19
train, Loss: 9.720127838